# Examples of usage

In [1]:
import numpy as np
import logging

In [2]:
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [4]:
from core import WMFileSource, PythonSource, MNPSolver
from mnp import (GurobiAlgorithm, GreedyAlgorithm, SimulatedAnnealing,
                transition_greedy_n, transition_greedy_one, temperature_div,
                temperature_div_log, transition_greedy_many)


# Small Instance

In [5]:
ext_s_one = PythonSource(
    data=[8, 6, 7, 4, 5],
    item_type=float,
    to_collection=(lambda x: np.array(list(x))),
)

## Greedy Algorithm

In [6]:
solver = MNPSolver(source=ext_s_one, algorithm=GreedyAlgorithm(number_of_sets=3))
greedy_of, greedy_sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.PythonSource object at 0x7f9cab7fb580>
ALGORITHM: <mnp.greedy.GreedyAlgorithm object at 0x7f9cab7fb970>
OF: 6.0
SETS:
SET #0: [8]
SET #1: [7, 4]
SET #2: [6, 5]
SET SUMS: 8 11 11

## Simulated Annealing

In [7]:
solver = MNPSolver(
    source=ext_s_one,
    algorithm=SimulatedAnnealing(
        start_state=[[8, 7, 4, 6, 5], [], []],
        transition_func=transition_greedy_n(7),
        temperature_func=temperature_div_log,
        t_max=10**6,
        t_min=10**-12,
        max_iter=10**2,
        number_of_sets=3,
    )
)
_of, _sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.PythonSource object at 0x7f9cab7fb580>
ALGORITHM: <mnp.simulated_annealing.SimulatedAnnealing object at 0x7f9cab811100>
OF: 6.0
SETS:
SET #0: [7, 4]
SET #1: [8]
SET #2: [6, 5]
SET SUMS: 11 8 11

## Gurobi

In [8]:
solver = MNPSolver(source=ext_s_one, algorithm=GurobiAlgorithm(number_of_sets=3))
_of, _sets = solver.solve_problem()
solver

Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 15 columns and 15 nonzeros
Model fingerprint: 0x255c147a
Model has 45 quadratic objective terms
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 2e+02]
  QObjective range [3e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 600.0000000
Presolve time: 0.00s
Presolved: 35 rows, 45 columns, 105 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective -1.100000e+02, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -110.00000    0    8 

MNP SOLVER INSTANCE
SOURCE: <core.data_source.PythonSource object at 0x7f9cab7fb580>
ALGORITHM: <mnp.exact.GurobiAlgorithm object at 0x7f9cab811700>
OF: 6.0
SETS:
SET #0: [7, 4]
SET #1: [6, 5]
SET #2: [8]
SET SUMS: 11 11 8

# Big instance

In [9]:
wms = WMFileSource(
    filepath='instances/data.txt',
    item_type=int,
    to_collection=(lambda x: np.array(list(x))),
)

## Greedy Algorithm

In [10]:
solver = MNPSolver(source=wms, algorithm=GreedyAlgorithm(number_of_sets=20))
greedy_of, greedy_sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.WMFileSource object at 0x7f9cab7fb4f0>
ALGORITHM: <mnp.greedy.GreedyAlgorithm object at 0x7f9cab811220>
OF: 7612188.0
SETS:
SET #0: [387162, 201347, 89243, 81049, 69547, 60230, 51051, 45444, 30247, 23256, 16676]
SET #1: [375698, 206707, 89859, 81724, 71640, 65415, 50152, 38229, 37140, 21309, 18793]
SET #2: [372906, 208596, 90314, 82469, 70866, 65893, 49903, 38230, 38206, 20440, 19311]
SET #3: [357826, 210543, 95292, 88151, 71789, 68027, 49046, 40868, 34194, 23080, 14485, 1933, 1302]
SET #4: [349541, 211791, 99148, 88870, 80539, 55712, 53662, 46912, 27698, 26581, 9948, 6415]
SET #5: [346671, 219218, 98418, 86760, 78372, 56643, 52539, 47239, 27707, 27196, 8993, 7059]
SET #6: [342797, 228372, 92616, 87745, 73494, 66449, 49418, 40264, 34999, 22385, 16959]
SET #7: [339311, 234039, 92018, 84625, 78777, 59258, 51914, 44558, 30807, 23523, 14191, 3606]
SET #8: [339195, 240534, 91393, 83809, 69933, 66543, 49526, 39093, 35133, 25408, 9152, 7314]
SET #

## Simulated Annealing

In [11]:
solver = MNPSolver(
    source=wms,
    algorithm=SimulatedAnnealing(
        start_state=greedy_sets,
        transition_func=transition_greedy_many(5),
        temperature_func=temperature_div_log,
        t_max=10**5,
        t_min=0,
        max_iter=10**5,
        number_of_sets=20,
    )
)
_of, _sets = solver.solve_problem()
solver

/home/spefk/programming/mnp_algorithms/mnp/simulated_annealing.py:113: RuntimeWarning: overflow encountered in double_scalars
  if of_cur > of_new or random.random() < np.exp(-(of_new - of_cur) / t_cur):


MNP SOLVER INSTANCE
SOURCE: <core.data_source.WMFileSource object at 0x7f9cab7fb4f0>
ALGORITHM: <mnp.simulated_annealing.SimulatedAnnealing object at 0x7f9cab811310>
OF: 4912642.0
SETS:
SET #0: [201347, 81049, 45444, 60230, 89243, 51051, 387162, 30247, 16676, 23256, 70270]
SET #1: [18793, 71640, 37140, 65415, 21309, 206707, 375698, 81724, 38229, 89859, 49418]
SET #2: [49903, 19311, 20440, 38206, 38230, 82469, 208596, 90314, 70866, 372906, 65893]
SET #3: [34194, 88151, 210543, 357826, 14485, 95292, 23080, 1471, 68027, 49046, 1302, 71789, 40868]
SET #4: [6415, 26581, 211791, 349541, 27698, 9948, 80539, 53662, 55712, 99148, 46912, 88870]
SET #5: [346671, 56643, 27196, 78372, 47239, 98418, 7059, 86760, 52164, 219218, 8993, 27707]
SET #6: [34999, 342797, 66449, 40264, 16959, 87745, 50152, 92616, 228372, 22385, 73494]
SET #7: [51914, 92018, 78777, 234039, 44558, 59258, 3606, 339311, 14191, 30733, 84625, 23523]
SET #8: [25408, 240534, 91393, 49526, 69933, 35133, 83809, 39093, 9152, 339195, 66

## Tabu Search

## Ant Colony Optimization


# Benchmarking

In [ ]:
# TODO: Table Algorithm: (OF, Time)

In [ ]:
# TODO: Plot N/Time